# Inference with the YOLO Brain MRI Service

In [1]:
from PIL import Image
import numpy as np
import requests
import json
import os
import torch
from ultralytics.utils.ops import non_max_suppression
import matplotlib.pyplot as plt

In [2]:
def preprocess(image):
    image = Image.open(image)
    image = image.resize((256, 256))

    # BGR to RGB, BHWC to BCHW, (n, 3, h, w)
    image = np.array(image)[:, :, :3]
    image = image[:, :, ::-1]
    image = image.transpose((2, 0, 1))
    image = image[np.newaxis, :, :, :]
    image = torch.tensor(image.copy(), dtype=torch.float32)
    image = image / 255.0 
      
    # Create request message to be sent to the predictor
    message_data = {}
    inputs = {}
    message_data["inputs"] = []
    inputs["name"] = "images"
    inputs["shape"] = image.shape
    inputs["datatype"] = "FP32"  # as the given onnx model expects float32
    inputs["data"] = image.tolist()
    message_data["inputs"].append(inputs)
    
    return message_data

        

In [3]:
# Configure Variables for the Predictor URL and Image Directory
image_dir = "yolo/datasets/yolo_mri_brain/test/images"
predictor_url = f"https://brain-mri-predictor-workshop.10-101-20-33.sslip.io/v2/models/brain-mri/infer"

In [4]:
# get images from the image directory
images = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".tif")]

# choose 5 random images
import random
images = random.sample(images, 50)

In [5]:
predictions = []
for image in images:
    # preprocess image
    message_data = preprocess(image)

    # Call predictor
    request_headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    response = requests.post(
        predictor_url, headers=request_headers, data=json.dumps(message_data), verify=False
    )
    response_message = json.loads(response.text)
    # convert list to tensor
    output = torch.tensor(response_message["outputs"][0]["data"]).reshape(response_message["outputs"][0]["shape"])
    # Apply non-max suppression
    prediction = non_max_suppression(prediction=output, conf_thres=0.5, iou_thres=0.5, classes=0)
    predictions.append(prediction[0])
    

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brain-mri-predictor-workshop.10-101-20-33.sslip.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brain-mri-predictor-workshop.10-101-20-33.sslip.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'brain-mri-predictor-workshop.10-101-20-33.sslip.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

In [ ]:
# plot images in a grid
num_im_per_row = 5
num_rows = len(images) // num_im_per_row
fig, axs = plt.subplots(num_rows, num_im_per_row, figsize=(num_im_per_row*5, num_rows*5))
for i, ax in enumerate(axs.flatten()):
    ax.imshow(Image.open(images[i]))
    ax.axis('off')
    ax.set_title(f"Prediction: {predictions[i].shape[0]}")
    # plot bounding boxes
    for box in predictions[i]:
        x1, y1, x2, y2 = box[:4]
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        # add confidence score
        ax.text(x1, y1, f"{box[4]:.2f}", color='red')
plt.show()
 
